In [ ]:
import streamlit as st
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# Load the models
cnn_model = tf.keras.models.load_model('cnn_model.h5')
vgg16_model = tf.keras.models.load_model('vgg16_model.h5')

# Load training history if available
cnn_history = np.load('cnn_history.npy', allow_pickle=True).item()
vgg16_history = np.load('vgg16_history.npy', allow_pickle=True).item()

# Function to preprocess the image
def preprocess_image(image):
    image = image.resize((224, 224))
    image = np.array(image)
    image = image / 255.0  # Normalize
    image = np.expand_dims(image, axis=0)  # Expand dimensions to fit the model input
    return image

# Function to plot loss and accuracy
def plot_metrics(history, model_name):
    fig, axs = plt.subplots(1, 2, figsize=(12, 4))
    
    # Plot loss
    axs[0].plot(history['loss'], label='Training Loss')
    axs[0].plot(history['val_loss'], label='Validation Loss')
    axs[0].set_title(f'{model_name} Loss')
    axs[0].set_xlabel('Epoch')
    axs[0].set_ylabel('Loss')
    axs[0].legend()
    
    # Plot accuracy
    axs[1].plot(history['accuracy'], label='Training Accuracy')
    axs[1].plot(history['val_accuracy'], label='Validation Accuracy')
    axs[1].set_title(f'{model_name} Accuracy')
    axs[1].set_xlabel('Epoch')
    axs[1].set_ylabel('Accuracy')
    axs[1].legend()

    st.pyplot(fig)

# App title
st.title('Image Classification Using Neural Networks')

# Image upload interface
uploaded_file = st.file_uploader("Upload an image for classification...", type=["jpg", "png"])

if uploaded_file is not None:
    image = Image.open(uploaded_file)
    st.image(image, caption='Uploaded Image.', use_column_width=True)
    
    # Model selection interface
    model_choice = st.selectbox("Select a model:", ("CNN Model", "VGG16 Model"))
    
    # Choose the model based on user selection
    if model_choice == "CNN Model":
        model = cnn_model
        history = cnn_history
    else:
        model = vgg16_model
        history = vgg16_history

    # Preprocess the image
    preprocessed_image = preprocess_image(image)
    
    # Perform prediction
    predictions = model.predict(preprocessed_image)
    predicted_class = np.argmax(predictions, axis=1)
    
    # Display the results
    st.write(f"Predicted Class: {predicted_class[0]}")
    st.write("Class Probabilities:")
    st.write(predictions)

    # Plot loss and accuracy graphs
    plot_metrics(history, model_choice)


In [ ]:
bash

streamlit run app.py